In [122]:
import pandas as pd

In [2]:
tmap = pd.read_csv('../data/Tmap_2005.csv')

In [71]:
c1 = tmap['지역(시도)'] == '서울'
select_list = ['공공기관', '의료시설', '연구기관', '업종별기업', '행정기관', '종교', '가정의례']
# c2 = tmap['지역(시군구)'] == '강남구'
# c2 = tmap['일자']==20200510
seoul = tmap.loc[c1,:]
seoul.shape

(23250, 8)

In [169]:
l = []
for i in select_list:
    select_condition2 = tmap['검색지유형2']==i
    seoul_index = seoul.loc[select_condition2,:].index
    # l.append(seoul_index)
    l.extend(seoul_index)
new_seoul = seoul.drop(l)
new_seoul['검색지유형3'].unique()
# seoul['검색지유형2'].unique()
# seoul['검색지유형3'].unique()

array(['아파트브랜드', '지하철역', '아파트단지', '백화점', '아파트건설사', '공원', '할인점', '주상복합',
       '농수축산물시장', '대학교', '쇼핑센터', '테마파크', '주요빌딩', '일반상가', '산업시설기타',
       '고속도로 IC', '오피스텔', '스케이트장', '기차역', '전망대', '주차장', '박물관/기념관', '호텔',
       '전자제품양판점', '도서관', '극장', '커피전문점', '고등학교', '종합시장', '서적브랜드', '전문상가',
       '한식', '미술관', '드라이브코스', '문화유적지', '고궁', '버스터미널', '초등학교', '전시장',
       '수족관', '부페', '공항', '농수축산물점', '교차로', '학교내시설물', '유원지', '재래시장', '방송사',
       '운동장/체육관', '슈퍼마켓', '공연장', '섬', '분식', '기념비', '중학교', '아파트형공장',
       '종합스포츠센타', '골프장', '대한상공회의소', '은행', '주요건물기타', '주유소', '자동차영업소',
       '중고차매매', '대교/철교', '사회복지시설', '화랑/전시관', '삼성', '센터', '능묘', '회관',
       '도시고속화도로IC', '축제', '운전면허시험장', 'LG유플러스', '호수', '한샘가구', '관광안내소',
       '장난감', '동식물원', '패밀리레스토랑', '놀이시설'], dtype=object)

In [170]:
# 서울시 구
gu_list = new_seoul['지역(시군구)'].unique()
# 점수주기
new_seoul['point'] = 31-new_seoul['검색지랭킹(시군구내)']
# 지역별 점수 합산
result = pd.DataFrame()
for i in gu_list:
    con_gu = new_seoul['지역(시군구)'] == i
    # print(i)
    gu_seoul = new_seoul.loc[con_gu, :]
    s = gu_seoul.groupby(by=['검색지명']).sum()
    s = s.loc[s['point']>20,:]
    ss = s.sort_values(by='point', ascending=False).reset_index()
    ss['gu'] = i
    result=result.append(ss[['검색지명', 'point', 'gu']])
result.head(40)
gu_list

array(['강동구', '성동구', '송파구', '양천구', '영등포구', '용산구', '종로구', '중랑구', '강남구',
       '강서구', '관악구', '광진구', '동작구', '마포구', '서초구', '은평구', '강북구', '구로구',
       '노원구', '도봉구', '동대문구', '성북구', '중구', '금천구', '서대문구'], dtype=object)

In [153]:
cnt_seoul = seoul['검색지명'].value_counts()

In [154]:
type(cnt_seoul)
data = {'name':[], 'count':[]}
for idx, i in enumerate(cnt_seoul.index):
    if cnt_seoul[idx]<=1: break
    data['name'].append(i)
    data['count'].append(cnt_seoul[idx])
data
cnt_df = pd.DataFrame(data)
# cnt_df.head(50)
# 음 한달동안 서울에서 20회 이하는 삭제했을때 

# 카카오 api
- 카카오 api 키워드로 장소검색을 사용하면 각각의 네비게이션 목적지들의 좌표값이 나온다.
- 지하철 역일경우에는 인구 분포가 안나오는데 그외에는 나온다.

In [8]:
# cnt_df.plot(kind='bar')
import requests
from decouple import config
ka_key = config('KAKAO_RESTAPI')
kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=광운대학교'
h = {
    'Authorization': 'KakaoAK '+ka_key
}
res = requests.get(kakao_url, headers=h)
res.json()

{'documents': [{'address_name': '서울 노원구 월계동 447-1',
   'category_group_code': 'SC4',
   'category_group_name': '학교',
   'category_name': '교육,학문 > 학교 > 대학교',
   'distance': '',
   'id': '10148561',
   'phone': '02-940-5114',
   'place_name': '광운대학교',
   'place_url': 'http://place.map.kakao.com/10148561',
   'road_address_name': '서울 노원구 광운로 20',
   'x': '127.058270608867',
   'y': '37.6192404638865'},
  {'address_name': '서울 노원구 월계동 466',
   'category_group_code': '',
   'category_group_name': '',
   'category_name': '교육,학문 > 학교부속시설',
   'distance': '',
   'id': '17564275',
   'phone': '02-940-8614',
   'place_name': '광운대학교 동해문화예술관',
   'place_url': 'http://place.map.kakao.com/17564275',
   'road_address_name': '서울 노원구 광운로 21',
   'x': '127.057577863828',
   'y': '37.6198462676801'},
  {'address_name': '서울 노원구 월계동 466',
   'category_group_code': 'CT1',
   'category_group_name': '문화시설',
   'category_name': '문화,예술 > 문화시설 > 공연장,연극극장',
   'distance': '',
   'id': '1817730210',
   'phone': '',

# 전체지역

In [21]:
cnt_tmap = tmap['검색지명'].value_counts()
cnt_tmap

솔내음          95
수림정          93
목월빵집         93
하우스베이커리      93
이성당          93
             ..
영덕읍행정복지센터     1
재능중학교         1
한일합섬의령공장      1
서영여자고등학교      1
구미시보건소        1
Name: 검색지명, Length: 16307, dtype: int64